In [ ]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
import numpy as np
import os
import random
import tensorflow as tf
from PIL import Image #读取图像处理代码
import os.path
import glob

C:\Users\zhangmy3\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

def convertjpg(jpgfile,outdir,width=256,height=256):
    img=Image.open(jpgfile)
    try:
        new_img=img.resize((width,height),Image.BILINEAR)   
        new_img.save(os.path.join(outdir,os.path.basename(jpgfile)))
    except Exception as e:
        print(e)
for jpgfile in glob.glob("../images/full/*.jpg"):
    convertjpg(jpgfile,"../images/full1/")


In [3]:
# Get images
X = []
for filename in os.listdir('../images/full1/'):
    X.append(img_to_array(load_img('../images/full1/'+filename)))
image = np.array(X, dtype=float)
image = np.array(image, dtype=float)

(360, 256, 256, 3)

In [5]:
X = rgb2lab(1.0/255*image)[:,:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,:,1:]
Y /= 128
X = X.reshape(110, 256, 256, 1)
Y = Y.reshape(110, 256, 256, 2)

In [6]:
# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [7]:
# Finish model
model.compile(optimizer='rmsprop',
			loss='mse')

In [8]:
model.fit(x=X, 
	y=Y,
	batch_size=5,
	epochs=100)

Epoch 1/100
110/110 [==============================] - 19s 175ms/step - loss: 0.1715
Epoch 2/100
110/110 [==============================] - 18s 167ms/step - loss: 0.0411
Epoch 3/100
110/110 [==============================] - 18s 166ms/step - loss: 0.0395
Epoch 4/100
110/110 [==============================] - 19s 172ms/step - loss: 0.0385
Epoch 5/100
110/110 [==============================] - 19s 176ms/step - loss: 0.0378
Epoch 6/100
110/110 [==============================] - 18s 167ms/step - loss: 0.0374
Epoch 7/100
110/110 [==============================] - 19s 168ms/step - loss: 0.0371
Epoch 8/100
110/110 [==============================] - 19s 177ms/step - loss: 0.0369
Epoch 9/100
110/110 [==============================] - 18s 168ms/step - loss: 0.0368
Epoch 10/100
110/110 [==============================] - 19s 168ms/step - loss: 0.0368
Epoch 11/100
110/110 [==============================] - 19s 172ms/step - loss: 0.0367
Epoch 12/100
110/110 [==============================] - 21s 190

In [15]:
image = img_to_array(load_img('../images/test/e8e9bbcd94ed1b011456d4560e24bdd9fef995a6.jpg'))
image = np.array(image, dtype=float)
X = rgb2lab(1.0/255*image)[:,:,0]
X = X.reshape(1, 256, 256, 1)


ValueError: Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (239, 300)

In [8]:

output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((256, 256, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))


C:\Users\zhangmy3\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\color\colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 1671 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
C:\Users\zhangmy3\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\zhangmy3\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


C:\Users\zhangmy3\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\zhangmy3\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\color\colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 27 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)


AttributeError: module 'tensorflow' has no attribute 'device'